# Getting started with Rhapsody

## Installation
Follow instructions in the git repository [README file](https://github.com/luponzo86/rhapsody/blob/master/README.md).

## Initial configuration

In [1]:
import rhapsody as rd

In [2]:
rd.initialSetup()

@> Pre-existing working directory detected: /home/lponzoni/rhapsody
@> Pre-existing classifiers found: /home/lponzoni/rhapsody/default_classifiers-sklearn_v0.20.3
@> EVmutation folder found: /home/lponzoni/rhapsody/EVmutation_mutation_effects
@> DSSP is installed on the system
@> Setup complete


## Testing

In [3]:
import os
import shutil

if os.path.isdir('local'):
    shutil.rmtree('local')
os.mkdir('local')

In [4]:
test_SAVs = ['O00294 496 A T',  # know neutral SAV used for training
             'O00238 31 R H',   # SAV with no PDB structure
             'P01112 58 T R',   # unknown SAV
             'P01112 30 D E',   # unknown SAV
             'P01112 170 K I',  # unknown SAV with no Pfam domain
             ]

In [5]:
os.chdir('local')

rh = rd.rhapsody(test_SAVs)

os.chdir('..')

@> Logging into file: rhapsody-log.txt
@> Logging started at 2019-06-20 17:51:29.337616
@> Imported feature set: 'wt_PSIC'
@>                       'Delta_PSIC'
@>                       'SASA'
@>                       'ANM_MSF-chain'
@>                       'ANM_effectiveness-chain'
@>                       'ANM_sensitivity-chain'
@>                       'stiffness-chain'
@>                       'entropy'
@>                       'ranked_MI'
@>                       'BLOSUM'
@> Submitting query to PolyPhen-2...
@> Query to PolyPhen-2 started in 25.7s.
@> PolyPhen-2 is running...
@> Query to PolyPhen-2 completed in 0.0s.
@> PolyPhen-2's output parsed.
@> Sequence-conservation features have been retrieved from PolyPhen-2's output.
@> Mapping SAVs to PDB structures...
@> [1/5] Mapping SAV 'O00238 31 R H' to PDB...
@> Pickle 'UniprotMap-O00238.pkl' recovered.
@> [2/5] Mapping SAV 'O00294 496 A T' to PDB...
@> Pickle 'UniprotMap-O00238.pkl' saved.
@> Pickle 'UniprotMap-O00294.pkl' recove

In [6]:
rh.mixPreds

array([(0.11266667, 0.05765328, 'neutral', 'known_neu'),
       (       nan,        nan, '?', 'new'),
       (0.95133334, 0.9392126 , 'deleterious', 'new'),
       (0.126     , 0.06226705, 'neutral', 'new'),
       (0.426     , 0.26697505, 'neutral', 'new')],
      dtype=[('score', '<f4'), ('path. probability', '<f4'), ('path. class', '<U12'), ('training info', '<U12')])